# Temporal Neighbourhood Coding (TNC) + CNN PFF

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:

from datetime import datetime

import lightning as L
import torch

from minerva.pipelines.lightning_pipeline import SimpleLightningPipeline

import torchmetrics

from minerva.data.data_modules.har import MultiModalHARSeriesDataModule
from minerva.analysis.metrics.balanced_accuracy import BalancedAccuracy
from minerva.analysis.model_analysis import TSNEAnalysis
from minerva.models.nets.time_series.cnns import CNN_PF_Backbone

from datetime import datetime

import lightning as L
import torch


from datetime import datetime

import lightning as L
import torch

from minerva.models.ssl.tnc import TNC
from minerva.models.nets.tnc import Discriminator_TNC
from minerva.data.data_modules.har_xu_23 import HarDataModule
from datetime import datetime
import lightning as L
import torch
import torchmetrics
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

from minerva.analysis.metrics.balanced_accuracy import BalancedAccuracy
from minerva.analysis.model_analysis import TSNEAnalysis
from minerva.data.data_modules.har import MultiModalHARSeriesDataModule
from minerva.data.data_modules.har_rodrigues_24 import HARDataModuleCPC
from minerva.models.loaders import FromPretrained
from minerva.models.nets.base import SimpleSupervisedModel
from minerva.models.nets.cpc_networks import HARCPCAutoregressive
from minerva.models.nets.mlp import MLP
from minerva.models.nets.time_series.cnns import CNN_PF_Backbone
from minerva.models.ssl.cpc import CPC
from minerva.pipelines.lightning_pipeline import SimpleLightningPipeline


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
execution_id = f'run_{datetime.now().strftime("%Y%m%d-%H%M%S")}'
log_dir = f"./logs/{execution_id}" 

print(f"Execution ID: {execution_id}")
print(f"Log dir: {log_dir}")

Execution ID: run_20260127-023233
Log dir: ./logs/run_20260127-023233


## Pre-training with TNC

### 1. Defining Data Module

In [4]:
data_module = HarDataModule(
    # processed_data_dir="/workspaces/HIAAC-KR-Dev-Container/shared_data/authentication/standartized_balanced/RecodGait_v1_TNC",
    processed_data_dir="//workspaces/HIAAC-KR-Dev-Container/shared_data/xu_2023_datasets/1-1/kuhar/processed/",
    window_size=60,
    batch_size=16,
    adf=False,
)

data_module


### 2. Defining TNC Model

In [5]:
# Simple forward pass to check the output shape
data_module.setup("fit")
dataset = data_module.train_dataloader()
batch = next(iter(dataset))

# Create a backbone model
g_enc = CNN_PF_Backbone(include_middle=True, permute=True,flatten=True)
r = g_enc.forward(batch[0])
r.shape

torch.Size([16, 768])

In [6]:
model = TNC(
    backbone=g_enc,
    projection_head=Discriminator_TNC(
        # input_size=r.shape[-1],
        input_size=768,
        max_pool=False
    ),
    w=0.2
)

model

TNC(
  (backbone): CNN_PF_Backbone(
    (first_padder): ZeroPadder2D(pad_at=(3,), padding_size=2)
    (upper_part): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
    (lower_part): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
    (middle_part): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
    (shared_part): Sequential(
      (0): Conv2d(48, 64, kernel_size=(3, 5), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
  )
  (projection_head): Discriminator_TNC(
    (model): Se

### 3. Defining the Pytorch Lightning Trainer Configuration

In [7]:
## Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/',
    monitor='val_loss',
    mode='min',
    save_last=True
)

## Logger
logger = CSVLogger(save_dir=log_dir, name='tnc-cnn-pff-pretraining', version=execution_id)

## Trainer
trainer = L.Trainer(
    max_epochs=3,
    accelerator="gpu",
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # Only for testing. Remove for production. We will only train using 1 batch
    limit_train_batches=1,
    limit_val_batches=1,
)

trainer

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


### 4. Creating the training pipeline (and running the training)

In [8]:
train_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42
)
train_pipeline.run(data_module, task="fit")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


** Seed set to: 42 **
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/codecarbon/logs/run_20260127-023233/run_2026-01-27-02-32-3409fc2a17.yaml



  | Name            | Type              | Params | Mode 
--------------------------------------------------------------
0 | backbone        | CNN_PF_Backbone   | 46.6 K | train
1 | projection_head | Discriminator_TNC | 4.7 M  | train
2 | loss_fn         | BCEWithLogitsLoss | 0      | train
--------------------------------------------------------------
4.8 M     Trainable params
0         Non-trainable params
4.8 M     Total params
19.085    Total estimated model params size (MB)
25        Modules in train mode
0         Modules in eval mode


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s] 


 Logging timestamp - Train Loss: 0.6848717927932739,

 
Epoch 1:   0%|          | 0/1 [00:00<?, ?it/s, v_num=3233]        


 Logging timestamp - Train Loss: 0.6827762722969055,

 
Epoch 2:   0%|          | 0/1 [00:00<?, ?it/s, v_num=3233]        


 Logging timestamp - Train Loss: 0.6871581077575684,

 
Epoch 2: 100%|██████████| 1/1 [00:00<00:00,  1.14it/s, v_num=3233]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 1/1 [00:01<00:00,  0.50it/s, v_num=3233]
⏱️ fit took 8.25s → saved to /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/codecarbon/logs/run_20260127-023233/timings_fit.csv
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/codecarbon/logs/run_20260127-023233/run_2026-01-27-02-32-3409fc2a17.yaml


### 5. Inspecting checkpoints

In [9]:
ckpt_path = checkpoint_callback.last_model_path
ckpt = torch.load(ckpt_path, map_location="cuda")
ckpt = ckpt.get("state_dict", ckpt)
list(ckpt.keys())

['backbone.upper_part.0.weight',
 'backbone.upper_part.0.bias',
 'backbone.lower_part.0.weight',
 'backbone.lower_part.0.bias',
 'backbone.middle_part.0.weight',
 'backbone.middle_part.0.bias',
 'backbone.shared_part.0.weight',
 'backbone.shared_part.0.bias',
 'projection_head.model.0.weight',
 'projection_head.model.0.bias',
 'projection_head.model.3.weight',
 'projection_head.model.3.bias']

## Fine-tunning with CNN PFF

### 1. Defining the Data Module

In [10]:
data_module = MultiModalHARSeriesDataModule(
    data_path="/workspaces/HIAAC-KR-Dev-Container/shared_data/daghar/standardized_view/MotionSense/",
    feature_prefixes=["accel-x", "accel-y", "accel-z", "gyro-x", "gyro-y", "gyro-z"],
    label="standard activity code",
    features_as_channels=True,
    cast_to="float32",
    batch_size=64,
)

data_module

MultiModalHARSeriesDataModule(data_path=/workspaces/HIAAC-KR-Dev-Container/shared_data/daghar/standardized_view/MotionSense, batch_size=64)

In [11]:
# Pega os dataloaders de treino e validação
data_module.setup("fit")
train_data_loader = data_module.train_dataloader()
validation_data_loader = data_module.val_dataloader()
first_batch = next(iter(train_data_loader))

X, y = first_batch
print(X.shape, y.shape)

Using DataLoader with shuffle=True
Using DataLoader with shuffle=False
torch.Size([64, 6, 60]) torch.Size([64])


### 2. Defining the CNN PFF Model for Fine-tuning

In [12]:
backbone = CNN_PF_Backbone(include_middle=True, permute=False,flatten=True)
backbone = FromPretrained(
    model=backbone, 
    ckpt_path=checkpoint_callback.best_model_path,
    filter_keys=["backbone"],
    keys_to_rename={"backbone.": ""},      # Let's remove the prefix from the keys
                                        # on the checkpoint to load the model
                                        # correctly
    strict=True,
    error_on_missing_keys=True
)

Performing key renaming with: {'backbone.': ''}
	Renaming key: backbone.upper_part.0.weight -> upper_part.0.weight (changed: True)
	Renaming key: backbone.upper_part.0.bias -> upper_part.0.bias (changed: True)
	Renaming key: backbone.lower_part.0.weight -> lower_part.0.weight (changed: True)
	Renaming key: backbone.lower_part.0.bias -> lower_part.0.bias (changed: True)
	Renaming key: backbone.middle_part.0.weight -> middle_part.0.weight (changed: True)
	Renaming key: backbone.middle_part.0.bias -> middle_part.0.bias (changed: True)
	Renaming key: backbone.shared_part.0.weight -> shared_part.0.weight (changed: True)
	Renaming key: backbone.shared_part.0.bias -> shared_part.0.bias (changed: True)
Model loaded from /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/codecarbon/checkpoints/epoch=0-step=1.ckpt


In [13]:
# Dummy forward pass to check the output shape
r = backbone.forward(X)
r.shape

torch.Size([64, 768])

In [14]:
head = MLP([
    768,  # r.shape[1]*r.shape[2],  (64*12)
    128, 
    6
])
head

MLP(
  (0): Linear(in_features=768, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=6, bias=True)
)

In [15]:
model = SimpleSupervisedModel(
    backbone=backbone,
    fc=head,
    loss_fn=torch.nn.CrossEntropyLoss(),
    flatten=False,
    train_metrics={
        "acc": torchmetrics.Accuracy(task="multiclass", num_classes=6),
    },
    val_metrics={
        "acc": torchmetrics.Accuracy(task="multiclass", num_classes=6),
    },
)

model

SimpleSupervisedModel(
  (backbone): CNN_PF_Backbone(
    (first_padder): ZeroPadder2D(pad_at=(3,), padding_size=2)
    (upper_part): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
    (lower_part): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
    (middle_part): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
    (shared_part): Sequential(
      (0): Conv2d(48, 64, kernel_size=(3, 5), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
  )
  (fc): MLP(
    (0): Linear(in_featu

In [16]:
# Option 1: Measure ALL model parameters (including backbone AND head)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total model parameters: {total_params:,}")
print(f"Trainable model parameters: {trainable_params:,}")

# Option 2: Measure components separately (as you intended)
# Backbone parameters
backbone_total = sum(p.numel() for p in model.backbone.parameters())
backbone_trainable = sum(p.numel() for p in model.backbone.parameters() if p.requires_grad)

# Head parameters (use model.fc, not head variable)
head_total = sum(p.numel() for p in model.fc.parameters())
head_trainable = sum(p.numel() for p in model.fc.parameters() if p.requires_grad)

print(f"\nBackbone:")
print(f"  Total parameters: {backbone_total:,}")
print(f"  Trainable parameters: {backbone_trainable:,}")

print(f"\nClassification Head:")
print(f"  Total parameters: {head_total:,}")
print(f"  Trainable parameters: {head_trainable:,}")

# Verify the sum matches Option 1
print(f"\nVerification:")
print(f"Backbone + Head total: {backbone_total + head_total:,}")
print(f"Should match total: {total_params:,}")

from thop import profile
evaluation_data = torch.rand(1000, 6, 60, device='cuda')
model.to('cuda')
macs, params = profile(model, inputs=(evaluation_data,))

print(f"MACs: {macs:,}")
print(f"Parameters: {params:,}")

# from codecarbon import EmissionsTracker

torch.cuda.is_available()
torch.cuda.device_count()






Total model parameters: 145,830
Trainable model parameters: 145,830

Backbone:
  Total parameters: 46,624
  Trainable parameters: 46,624

Classification Head:
  Total parameters: 99,206
  Trainable parameters: 99,206

Verification:
Backbone + Head total: 145,830
Should match total: 145,830
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
MACs: 1,673,856,000.0
Parameters: 145,830.0


1

In [17]:
import pandas as pd
from codecarbon import EmissionsTracker

TOTAL_RUNS = 14
DISCARD_FIRST = 2
DISCARD_LAST = 2

results = []

tracker = EmissionsTracker(
    project_name="basic_measurement",
    measure_power_secs=10,
    save_to_file=False
)

try:
    for run_id in range(TOTAL_RUNS):
        tracker.start_task(f"measure_inference_{run_id}")

        _ = model(evaluation_data)  # inference

        emissions = tracker.stop_task()

        energy_kwh = emissions.energy_consumed
        energy_mwh = energy_kwh * 1_000          # kWh → mWh
        energy_j   = energy_kwh * 3_600_000      # kWh → J
        emissions_g = emissions.emissions * 1_000 * 1_000  # kg → g→ mg

        results.append({
            "run": run_id,
            "energy_mWh": energy_mwh,
            "energy_J": energy_j,
            "emissions_mgCO2eq": emissions_g,
            # "duration_s": emissions.duration
        })

finally:
    tracker.stop()

# --- All runs ---
df_all = pd.DataFrame(results)

# --- Valid runs (ignore first & last 2) ---
df_valid = df_all.iloc[DISCARD_FIRST: TOTAL_RUNS - DISCARD_LAST]

# --- Statistics ---
mean = df_valid[["energy_mWh", "energy_J", "emissions_mgCO2eq"]].mean()
std  = df_valid[["energy_mWh", "energy_J", "emissions_mgCO2eq"]].std()

summary = pd.DataFrame(
    [mean, std],
    index=["mean", "std"]
).round(2)

# --- Final table ---
df_final = pd.concat([
    df_valid.set_index("run").round(2),
    summary
])

df_final


[codecarbon WARNING @ 02:32:46] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 02:32:46] [setup] RAM Tracking...
[codecarbon INFO @ 02:32:46] [setup] CPU Tracking...
[codecarbon WARNING @ 02:32:47] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist, and are readable, at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 02:32:47] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU E5-2630 v2 @ 2.60GHz
[codecarbon WARNING @ 02:32:47] No CPU tracking mode found. Falling back on CPU load mode.
[codecarbon INFO @ 02:32:47] [setup] GPU Tracking...
[codecarbon INFO @ 02:32:47] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 02:32:47] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: cpu_load
                GPU Tracking Method: pynvml
         

,energy_mWh,energy_J,emissions_mgCO2eq
2,0.01,38.24,1.04
3,0.01,38.36,1.05
4,0.01,43.80,1.20
5,0.01,39.33,1.07
6,0.01,38.56,1.05
7,0.01,38.97,1.06
8,0.01,38.18,1.04
9,0.01,38.60,1.05
10,0.01,39.21,1.07
11,0.01,38.09,1.04


In [18]:
a

NameError: name 'a' is not defined

### 3. Defining the Pytorch Lightning Trainer Configuration

In [ ]:
## Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/',
    monitor='val_loss',
    mode='min',
    save_last=True
)

## Logger
logger = CSVLogger(save_dir=log_dir, name='tnc-cnn-pff-finetune', version=execution_id)

## Trainer
trainer = L.Trainer(
    max_epochs=3,
    accelerator="gpu",
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # Only for testing. Remove for production. We will only train using 1 batch
    limit_train_batches=1,
    limit_val_batches=1,
)

trainer

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


### 4. Creating the fine-tuning pipeline (and running the training)

In [ ]:
train_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42
)
train_pipeline.run(data_module, task="fit")

Seed set to 42





 cuda device: 


 NVIDIA RTX A5000
Log directory set to: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/logs/run_20250218-184817
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/logs/run_20250218-184817/run_2025-02-18-18-48-317ba4bd2be4bb4d56bc09042f0f3f441f.yaml


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]

  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | backbone | CNN_PF_Backbone  | 46.6 K | train
1 | fc       | MLP              | 99.2 K | train
2 | loss_fn  | CrossEntropyLoss | 0      | train
------------------------------------------------------
145 K     Trainable params
0         Non-trainable params
145 K     Total params
0.583     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]Using DataLoader with shuffle=False
Using DataLoader with shuffle=True                                         


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 1/1 [00:07<00:00,  0.14it/s, v_num=4817, val_loss=1.860, val_acc=0.000, train_loss=1.500, train_acc=0.203]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 1/1 [00:07<00:00,  0.14it/s, v_num=4817, val_loss=1.860, val_acc=0.000, train_loss=1.500, train_acc=0.203]
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/logs/run_20250218-184817/run_2025-02-18-18-48-317ba4bd2be4bb4d56bc09042f0f3f441f.yaml


## Evaluating the Fine-tuned Model

In [ ]:
test_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42,
    classification_metrics={
        "accuracy": torchmetrics.Accuracy(num_classes=6, task="multiclass"),
        "f1": torchmetrics.F1Score(num_classes=6, task="multiclass"),
        "precision": torchmetrics.Precision(num_classes=6, task="multiclass"),
        "recall": torchmetrics.Recall(num_classes=6, task="multiclass"),
        "balanced_accuracy": BalancedAccuracy(num_classes=6, task="multiclass"),
    },
    apply_metrics_per_sample=False,
    model_analysis={
        "tsne": TSNEAnalysis(
            height=800,
            width=800,
            legend_title="Activity",
            title="t-SNE of CPC Finetuned on MotionSense",
            output_filename="tsne_cpc_finetuned_motionsense.pdf",
            label_names={
                0: "sit",
                1: "stand",
                2: "walk",
                3: "stair up",
                4: "stair down",
                5: "run",
                6: "stair up and down",
            },
        )
    },
)

test_pipeline.run(
    data_module, task="evaluate", ckpt_path=checkpoint_callback.best_model_path
)

Seed set to 42





 cuda device: 


 NVIDIA RTX A5000
Log directory set to: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/logs/run_20250218-184817
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/logs/run_20250218-184817/run_2025-02-18-18-48-57fdd9388fc5334ddbbdcdbaa07d500e7a.yaml
Using DataLoader with shuffle=False


Restoring states from the checkpoint path at /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/checkpoints/epoch=1-step=2.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
Loaded model weights from the checkpoint at /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/checkpoints/epoch=1-step=2.ckpt


Using DataLoader with shuffle=False
Predicting DataLoader 0: 100%|██████████| 17/17 [00:00<00:00, 218.47it/s]
Running classification metrics...
Running model analysis...


/workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/minerva/analysis/metrics/balanced_accuracy.py:59: UserWarning: y_pred contains nan values and not all classes passed
  warnings.warn(f"y_pred contains nan values and not all classes passed")


Using DataLoader with shuffle=False


/workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/minerva/analysis/model_analysis.py:118: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  X = torch.tensor(X, device="cpu")


t-SNE plot saved to /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/logs/run_20250218-184817/tsne_cpc_finetuned_motionsense.pdf
Metrics saved to /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/logs/run_20250218-184817/metrics_2025-02-18-18-48-57fdd9388fc5334ddbbdcdbaa07d500e7a.yaml
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/logs/run_20250218-184817/run_2025-02-18-18-48-57fdd9388fc5334ddbbdcdbaa07d500e7a.yaml


{'classification': {'accuracy': [0.1666666716337204],
  'f1': [0.1666666716337204],
  'precision': [0.1666666716337204],
  'recall': [0.1666666716337204],
  'balanced_accuracy': [0.08349056541919708]},
 'analysis': {'tsne': '/workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tnc/logs/run_20250218-184817/tsne_cpc_finetuned_motionsense.pdf'}}